In [1]:
import fastf1 as ff1
import pandas as pd
import time

In [2]:
years = range(2018, 2026) # 2026 is excluded

In [3]:
# Info on the session
session = ff1.get_session(2021, 'Bahrain Grand Prix', 'R')
print(session.event)

req         WARNING 	DEFAULT CACHE ENABLED! (6.66 GB) C:\Users\vinor\AppData\Local\Temp\fastf1


RoundNumber                                                   1
Country                                                 Bahrain
Location                                                 Sakhir
OfficialEventName    FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021
EventDate                                   2021-03-28 00:00:00
EventName                                    Bahrain Grand Prix
EventFormat                                        conventional
Session1                                             Practice 1
Session1Date                          2021-03-26 14:30:00+03:00
Session1DateUtc                             2021-03-26 11:30:00
Session2                                             Practice 2
Session2Date                          2021-03-26 18:00:00+03:00
Session2DateUtc                             2021-03-26 15:00:00
Session3                                             Practice 3
Session3Date                          2021-03-27 15:00:00+03:00
Session3DateUtc                         

In [4]:
# Info on the year 2021
schedule = ff1.get_event_schedule(2021)
schedule

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Bahrain,Bahrain,FORMULA 1 ARAMCO PRE-SEASON TESTING 2021,2021-03-14,Pre-Season Test,testing,Practice 1,2021-03-12 10:00:00+03:00,2021-03-12 07:00:00,...,Practice 3,2021-03-14 10:00:00+03:00,2021-03-14 07:00:00,None,NaT,NaT,None,NaT,NaT,False
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,2021-03-28,Bahrain Grand Prix,conventional,Practice 1,2021-03-26 14:30:00+03:00,2021-03-26 11:30:00,...,Practice 3,2021-03-27 15:00:00+03:00,2021-03-27 12:00:00,Qualifying,2021-03-27 18:00:00+03:00,2021-03-27 15:00:00,Race,2021-03-28 18:00:00+03:00,2021-03-28 15:00:00,True
2,2,Italy,Imola,FORMULA 1 PIRELLI GRAN PREMIO DEL MADE IN ITAL...,2021-04-18,Emilia Romagna Grand Prix,conventional,Practice 1,2021-04-16 11:00:00+02:00,2021-04-16 09:00:00,...,Practice 3,2021-04-17 11:00:00+02:00,2021-04-17 09:00:00,Qualifying,2021-04-17 14:00:00+02:00,2021-04-17 12:00:00,Race,2021-04-18 15:00:00+02:00,2021-04-18 13:00:00,True
3,3,Portugal,Portimão,FORMULA 1 HEINEKEN GRANDE PRÉMIO DE PORTUGAL 2021,2021-05-02,Portuguese Grand Prix,conventional,Practice 1,2021-04-30 11:30:00+01:00,2021-04-30 10:30:00,...,Practice 3,2021-05-01 12:00:00+01:00,2021-05-01 11:00:00,Qualifying,2021-05-01 15:00:00+01:00,2021-05-01 14:00:00,Race,2021-05-02 15:00:00+01:00,2021-05-02 14:00:00,True
4,4,Spain,Barcelona,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2021,2021-05-09,Spanish Grand Prix,conventional,Practice 1,2021-05-07 11:30:00+02:00,2021-05-07 09:30:00,...,Practice 3,2021-05-08 12:00:00+02:00,2021-05-08 10:00:00,Qualifying,2021-05-08 15:00:00+02:00,2021-05-08 13:00:00,Race,2021-05-09 15:00:00+02:00,2021-05-09 13:00:00,True
5,5,Monaco,Monte Carlo,FORMULA 1 GRAND PRIX DE MONACO 2021,2021-05-23,Monaco Grand Prix,conventional,Practice 1,2021-05-20 11:30:00+02:00,2021-05-20 09:30:00,...,Practice 3,2021-05-22 12:00:00+02:00,2021-05-22 10:00:00,Qualifying,2021-05-22 15:00:00+02:00,2021-05-22 13:00:00,Race,2021-05-23 15:00:00+02:00,2021-05-23 13:00:00,True
6,6,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2021,2021-06-06,Azerbaijan Grand Prix,conventional,Practice 1,2021-06-04 12:30:00+04:00,2021-06-04 08:30:00,...,Practice 3,2021-06-05 13:00:00+04:00,2021-06-05 09:00:00,Qualifying,2021-06-05 16:00:00+04:00,2021-06-05 12:00:00,Race,2021-06-06 16:00:00+04:00,2021-06-06 12:00:00,True
7,7,France,Le Castellet,FORMULA 1 EMIRATES GRAND PRIX DE FRANCE 2021,2021-06-20,French Grand Prix,conventional,Practice 1,2021-06-18 11:30:00+02:00,2021-06-18 09:30:00,...,Practice 3,2021-06-19 12:00:00+02:00,2021-06-19 10:00:00,Qualifying,2021-06-19 15:00:00+02:00,2021-06-19 13:00:00,Race,2021-06-20 15:00:00+02:00,2021-06-20 13:00:00,True
8,8,Austria,Spielberg,FORMULA 1 BWT GROSSER PREIS DER STEIERMARK 2021,2021-06-27,Styrian Grand Prix,conventional,Practice 1,2021-06-25 11:30:00+02:00,2021-06-25 09:30:00,...,Practice 3,2021-06-26 12:00:00+02:00,2021-06-26 10:00:00,Qualifying,2021-06-26 15:00:00+02:00,2021-06-26 13:00:00,Race,2021-06-27 15:00:00+02:00,2021-06-27 13:00:00,True
9,9,Austria,Spielberg,FORMULA 1 BWT GROSSER PREIS VON ÖSTERREICH 2021,2021-07-04,Austrian Grand Prix,conventional,Practice 1,2021-07-02 11:30:00+02:00,2021-07-02 09:30:00,...,Practice 3,2021-07-03 12:00:00+02:00,2021-07-03 10:00:00,Qualifying,2021-07-03 15:00:00+02:00,2021-07-03 13:00:00,Race,2021-07-04 15:00:00+02:00,2021-07-04 13:00:00,True


# General info

In [5]:
schedule = ff1.get_event_schedule(2025)
schedule

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Bahrain,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2025,2025-02-28,Pre-Season Testing,testing,Practice 1,2025-02-26 10:00:00+03:00,2025-02-26 07:00:00,...,Practice 3,2025-02-28 10:00:00+03:00,2025-02-28 07:00:00,None,NaT,NaT,None,NaT,NaT,True
1,1,Australia,Melbourne,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,2025-03-16,Australian Grand Prix,conventional,Practice 1,2025-03-14 12:30:00+11:00,2025-03-14 01:30:00,...,Practice 3,2025-03-15 12:30:00+11:00,2025-03-15 01:30:00,Qualifying,2025-03-15 16:00:00+11:00,2025-03-15 05:00:00,Race,2025-03-16 15:00:00+11:00,2025-03-16 04:00:00,True
2,2,China,Shanghai,FORMULA 1 HEINEKEN CHINESE GRAND PRIX 2025,2025-03-23,Chinese Grand Prix,sprint_qualifying,Practice 1,2025-03-21 11:30:00+08:00,2025-03-21 03:30:00,...,Sprint,2025-03-22 11:00:00+08:00,2025-03-22 03:00:00,Qualifying,2025-03-22 15:00:00+08:00,2025-03-22 07:00:00,Race,2025-03-23 15:00:00+08:00,2025-03-23 07:00:00,True
3,3,Japan,Suzuka,FORMULA 1 LENOVO JAPANESE GRAND PRIX 2025,2025-04-06,Japanese Grand Prix,conventional,Practice 1,2025-04-04 11:30:00+09:00,2025-04-04 02:30:00,...,Practice 3,2025-04-05 11:30:00+09:00,2025-04-05 02:30:00,Qualifying,2025-04-05 15:00:00+09:00,2025-04-05 06:00:00,Race,2025-04-06 14:00:00+09:00,2025-04-06 05:00:00,True
4,4,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2025,2025-04-13,Bahrain Grand Prix,conventional,Practice 1,2025-04-11 14:30:00+03:00,2025-04-11 11:30:00,...,Practice 3,2025-04-12 15:30:00+03:00,2025-04-12 12:30:00,Qualifying,2025-04-12 19:00:00+03:00,2025-04-12 16:00:00,Race,2025-04-13 18:00:00+03:00,2025-04-13 15:00:00,True
5,5,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2025,2025-04-20,Saudi Arabian Grand Prix,conventional,Practice 1,2025-04-18 16:30:00+03:00,2025-04-18 13:30:00,...,Practice 3,2025-04-19 16:30:00+03:00,2025-04-19 13:30:00,Qualifying,2025-04-19 20:00:00+03:00,2025-04-19 17:00:00,Race,2025-04-20 20:00:00+03:00,2025-04-20 17:00:00,True
6,6,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2025,2025-05-04,Miami Grand Prix,sprint_qualifying,Practice 1,2025-05-02 12:30:00-04:00,2025-05-02 16:30:00,...,Sprint,2025-05-03 12:00:00-04:00,2025-05-03 16:00:00,Qualifying,2025-05-03 16:00:00-04:00,2025-05-03 20:00:00,Race,2025-05-04 16:00:00-04:00,2025-05-04 20:00:00,True
7,7,Italy,Imola,FORMULA 1 AWS GRAN PREMIO DEL MADE IN ITALY E ...,2025-05-18,Emilia Romagna Grand Prix,conventional,Practice 1,2025-05-16 13:30:00+02:00,2025-05-16 11:30:00,...,Practice 3,2025-05-17 12:30:00+02:00,2025-05-17 10:30:00,Qualifying,2025-05-17 16:00:00+02:00,2025-05-17 14:00:00,Race,2025-05-18 15:00:00+02:00,2025-05-18 13:00:00,True
8,8,Monaco,Monaco,FORMULA 1 TAG HEUER GRAND PRIX DE MONACO 2025,2025-05-25,Monaco Grand Prix,conventional,Practice 1,2025-05-23 13:30:00+02:00,2025-05-23 11:30:00,...,Practice 3,2025-05-24 12:30:00+02:00,2025-05-24 10:30:00,Qualifying,2025-05-24 16:00:00+02:00,2025-05-24 14:00:00,Race,2025-05-25 15:00:00+02:00,2025-05-25 13:00:00,True
9,9,Spain,Barcelona,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2025,2025-06-01,Spanish Grand Prix,conventional,Practice 1,2025-05-30 13:30:00+02:00,2025-05-30 11:30:00,...,Practice 3,2025-05-31 12:30:00+02:00,2025-05-31 10:30:00,Qualifying,2025-05-31 16:00:00+02:00,2025-05-31 14:00:00,Race,2025-06-01 15:00:00+02:00,2025-06-01 13:00:00,True


## Number of races for year

In [6]:
total_races_from_2000 = 0
for year in years:
    try:
        schedule = ff1.get_event_schedule(year)
        print(f'{year}: {len(schedule)}')
        total_races_from_2000 += len(schedule)
    except:
        print(f'{year}: No data')

print(f'Total races in period 2000-2025: {total_races_from_2000}')


2018: 21
2019: 21
2020: 19
2021: 23
2022: 24
2023: 23
2024: 25
2025: 25
Total races in period 2000-2025: 181


## Info aggregated by pilot and laps

In [7]:
session = ff1.get_session(2024, 'French Grand Prix', 'R')
session.load()
session.laps


events      WARNING 	Correcting user input 'French Grand Prix' to 'Belgian Grand Prix'
core           INFO 	Loading data for Belgian Grand Prix - Race [v3.5.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '81', '16', '1', '4', '55', '11', '14', '31', '3', '18', '23', '10', '20', '77', '22'

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 00:57:07.735000,HAM,44,0 days 00:01:52.700000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:49.406000,...,True,Mercedes,0 days 00:55:14.764000,2024-07-28 13:03:52.742,1,2.0,False,,False,False
1,0 days 00:58:57.584000,HAM,44,0 days 00:01:49.849000,2.0,1.0,NaT,NaT,0 days 00:00:31.284000,0 days 00:00:49.211000,...,True,Mercedes,0 days 00:57:07.735000,2024-07-28 13:05:45.713,1,2.0,False,,False,True
2,0 days 01:00:46.693000,HAM,44,0 days 00:01:49.109000,3.0,1.0,NaT,NaT,0 days 00:00:31.215000,0 days 00:00:48.486000,...,True,Mercedes,0 days 00:58:57.584000,2024-07-28 13:07:35.562,1,1.0,False,,False,True
3,0 days 01:02:36.501000,HAM,44,0 days 00:01:49.808000,4.0,1.0,NaT,NaT,0 days 00:00:31.754000,0 days 00:00:48.605000,...,True,Mercedes,0 days 01:00:46.693000,2024-07-28 13:09:24.671,1,1.0,False,,False,True
4,0 days 01:04:25.877000,HAM,44,0 days 00:01:49.376000,5.0,1.0,NaT,NaT,0 days 00:00:31.592000,0 days 00:00:48.320000,...,True,Mercedes,0 days 01:02:36.501000,2024-07-28 13:11:14.479,1,1.0,False,,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,0 days 02:08:02.442000,RUS,63,0 days 00:01:47.631000,40.0,2.0,NaT,NaT,0 days 00:00:30.989000,0 days 00:00:47.555000,...,True,Mercedes,0 days 02:06:14.811000,2024-07-28 14:14:52.789,1,1.0,False,,False,True
837,0 days 02:09:50.196000,RUS,63,0 days 00:01:47.754000,41.0,2.0,NaT,NaT,0 days 00:00:30.926000,0 days 00:00:47.796000,...,True,Mercedes,0 days 02:08:02.442000,2024-07-28 14:16:40.420,1,1.0,False,,False,True
838,0 days 02:11:37.661000,RUS,63,0 days 00:01:47.465000,42.0,2.0,NaT,NaT,0 days 00:00:31.014000,0 days 00:00:47.525000,...,True,Mercedes,0 days 02:09:50.196000,2024-07-28 14:18:28.174,1,1.0,False,,False,True
839,0 days 02:13:24.962000,RUS,63,0 days 00:01:47.301000,43.0,2.0,NaT,NaT,0 days 00:00:30.990000,0 days 00:00:47.479000,...,True,Mercedes,0 days 02:11:37.661000,2024-07-28 14:20:15.639,1,1.0,False,,False,True


In [8]:
print(session.laps.Driver.value_counts())

Driver
HAM    44
PIA    44
LEC    44
VER    44
NOR    44
SAI    44
PER    44
ALO    44
OCO    44
RIC    44
STR    44
ALB    44
GAS    44
MAG    44
BOT    44
TSU    44
SAR    44
HUL    44
RUS    44
ZHO     5
Name: count, dtype: int64


# Cicle all the laps

In [9]:
years = range(2021, 2026) # 2026 is excluded

In [10]:
import warnings
import io
import sys
import logging
from contextlib import redirect_stdout, redirect_stderr

number_of_laps = 0
missing_data = []
warnings_log = {}  # Dictionary to store warnings by (year, event_name)
all_warning_lines = []  # List to store all lines containing "warning"

# Get the root logger to capture all logging output
root_logger = logging.getLogger()
original_level = root_logger.level
original_handlers = root_logger.handlers.copy()

for year in years:
    # Get the schedule for the year
    scheduled = ff1.get_event_schedule(year)

    # Select scheduled event that are races
    scheduled_race = scheduled[scheduled['Session5'] == 'Race'] # There are session that are not races

    # Get the event name
    event_names = scheduled_race['EventName'].values

    # Cicle through the events
    for event_name in event_names:
        print(f'--> {year}: {event_name}')
        
        session = ff1.get_session(year, event_name, 'R') # 'R' stands for race

        # Sometimes the data is not available
        try:
            # Setup log capture
            log_capture = io.StringIO()
            log_handler = logging.StreamHandler(log_capture)
            log_handler.setLevel(logging.WARNING)  # Capture WARNING and higher
            log_formatter = logging.Formatter('%(levelname)s - %(name)s - %(message)s')
            log_handler.setFormatter(log_formatter)
            
            # Temporarily modify logger to capture all warnings
            root_logger.setLevel(logging.WARNING)
            root_logger.handlers = [log_handler]
            
            # Also capture stdout/stderr for any direct prints
            stdout_capture = io.StringIO()
            stderr_capture = io.StringIO()
            
            # Load session with all captures in place
            with redirect_stdout(stdout_capture), redirect_stderr(stderr_capture), warnings.catch_warnings(record=True) as caught_warnings:
                warnings.simplefilter("always")
                try:
                    session.load()
                except Exception as e:
                    missing_data.append((year, event_name))
                    warnings_log[(year, event_name)] = {'error': str(e)}
            # Restore original logging configuration
            root_logger.setLevel(original_level)
            root_logger.handlers = original_handlers.copy()
            
            # Get captured outputs
            log_output = log_capture.getvalue()
            stdout_output = stdout_capture.getvalue()
            stderr_output = stderr_capture.getvalue()
            warning_messages = [str(w.message) for w in caught_warnings]
            
            # Check for warnings in log output
            log_lines = log_output.split('\n')
            for line in log_lines:
                if 'warning' in line.lower() or 'error' in line.lower():
                    all_warning_lines.append(f"{year} {event_name}: {line}")
            
            # Also check stdout and stderr
            for line in stdout_output.split('\n'):
                if 'warning' in line.lower() or 'error' in line.lower():
                    all_warning_lines.append(f"{year} {event_name}: {line}")
            
            for line in stderr_output.split('\n'):
                if 'warning' in line.lower() or 'error' in line.lower():
                    all_warning_lines.append(f"{year} {event_name}: {line}")
            
            # Store warnings if any were found
            if all_warning_lines:
                warnings_log[(year, event_name)] = {
                    'log_output': log_output,
                    'stdout': stdout_output, 
                    'stderr': stderr_output,
                    'python_warnings': warning_messages
                }
                print(f"  Captured warnings for {year} {event_name}")
                
        except Exception as e:
            missing_data.append((year, event_name))
            warnings_log[(year, event_name)] = {'error': str(e)}
            continue
        
        # Get the total number of laps in the session
        total_session_laps = session.laps.shape[0]
        number_of_laps += total_session_laps

        # Get drivers names
        drivers = session.laps.Driver.unique()

        # cicle through drivers
        for driver in drivers:
            pass

# Save all warning lines to a file
if all_warning_lines:
    with open('warning_lines.txt', 'w') as f:
        f.write('\n'.join(all_warning_lines))

    print(f"\nTotal warning lines: {len(all_warning_lines)}")
    print(f"Warning lines saved to 'warning_lines.txt'")
else:
    print("\nNo warning lines captured")

# For debugging: show a sample of the captured warnings
if all_warning_lines:
    print("\nSample of captured warnings:")
    for line in all_warning_lines[:5]:  # Show first 5 warnings
        print(line)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data


--> 2021: Bahrain Grand Prix


req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '77', '4', '11', '16', '3', '55', '22', '18', '7', '99', '31', '63', '5', '47', '10', '6', '14', '9']
core           INFO 	Loading data for Emilia Romagna Grand Prix - Race [v3.5.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timi

--> 2021: Emilia Romagna Grand Prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '44', '4', '16', '55', '3', '10', '18', '31', '14', '11', '22', '7', '99', '5', '47', '9', '77', '63', '6']
core           INFO 	Loading data for Portuguese Grand Prix - Race [v3.5.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


...

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

  Captured warnings for 2022 Japanese Grand Prix
--> 2022: United States Grand Prix


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
logger      WARNING 	Failed to load total lap count!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNING 	Failed to load track status data!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
logger      WARNING 	Failed to load timing data!
logger      WARNING 	Failed to add first lap 

  Captured warnings for 2022 United States Grand Prix


DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`

In [ ]:
print(f'Total number of laps in period 2000-2025: {number_of_laps}')

In [ ]:
session = ff1.get_session(2021, 'United States Grand Prix', 'R')
session.load()

core           INFO 	Loading data for United States Grand Prix - Race [v3.5.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
logger      WARNING 	Failed to load total lap count!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNING 	Failed to load track status data!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
logger      WARNING 	Failed to load timing data!
logger      WARNING 	Failed to add first lap time from Ergast!
req           

In [ ]:
session.laps

DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`